# Part 3 Solutions Notebook

## Introduction

Linen is a sustainable fabric made from cellulosic fibers derived from the stem of the flax plant.<sup>1</sup> Both the fabric and the fibers are named as linen. Linen fibers are much stronger (two to three times) than coton fibers and they dry at a much faster rate yielding cool, absorbent fabrics.<sup>2</sup> Due to its porous nature, linen is a good heat conductor and widely preferred for clothing and bedding in the summer.<sup>2</sup>

Linen is used heavily in clothing like dresses, suits, skirts etc. and in home fashion products like curtains, sheets and towels. Detection of defective end-products is highly valuable to manufacturers in those industries and human centered detection methods are highly costly and open to lots of flaws. In offline quality control method, one of the generally used methods, production is stopped to take samples of the product and carry out the tests.<sup>3</sup> Since the production stops for each sample, the cost of monitoring the process increases. Sophisticated quality control using advanced image processing techniques would significantly improve detection and provide manufacturers with enough time to adjust their processes with a lower cost.

Also, laundries, textile manufacturers, or chemical vendors may encounter with undesired outcomes in the wash process if the monitoring and process control of linen is not properly performed during the production.<sup>4</sup>

## Background Information

## Approach

The following libraries will be used in this part.

In [1]:
from PIL import Image
import statistics
import matplotlib.pyplot as plt
import numpy as np

## Results

## Conclusions and Future Work

## References

<sup>1</sup> https://textilelearner.blogspot.com/2012/01/linen-fiber-characteristics-of-linen.html

<sup>2</sup> https://www.thespruce.com/definition-of-linen-fabric-1976785

<sup>3</sup> https://www.textilemates.com/textile-testing-quality-control-fibre-testing-fabric-testing-yarn-testing

<sup>4</sup> https://www.almnet.org/page/TestPiece

> **NOTE:** The source code is available at [here](https://github.com/BU-IE-582/fall19-akinelden/blob/master/Project_Part3/Part3.ipynb) 